In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 4.4 MB 18.7 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 41.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 29.7 MB/s eta 0:00:01


In [3]:
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import numpy as np

In [4]:
class bertconfig():
    def __init__(self):
        self.project = 'THUC'
        self.bert_path = './bert_pretrain'
        self.tokenizer = BertTokenizer.from_pretrained(self.bert_path)
        self.class_list = [x.strip() for x in open('./THUC/data/class.txt').readlines()]
        self.pad_size = 32
        self.batch_size = 128
        self.device = 'cuda'
        self.hidden_size = 768
        self.num_classes = len(self.class_list)
        self.learning_rate = 1e-5
        self.num_epochs = 10
        self.save_path = './THUC/'+ self.project+'.ckpt'

In [5]:
config = bertconfig()

In [52]:
class Model(nn.Module): 

    def __init__(self, config):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(config.bert_path)
        self.fc = nn.Linear(config.hidden_size, config.num_classes)
        
    def forward(self, x):
        context = x[0].unsqueeze(0)  # 输入的句子
        mask = x[2].unsqueeze(0)  # mask
        _ = self.bert(context, attention_mask=mask)
        out = self.fc(_[1])
        prob = F.softmax(out)
        return prob

In [53]:
model = Model(config).to(config.device)

Some weights of the model checkpoint at ./bert_pretrain were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [54]:
model.load_state_dict(torch.load(config.save_path))

<All keys matched successfully>

In [55]:
def load_data(sentence,config):
    token = config.tokenizer.tokenize(sentence)
    token = ['CLS']+token
    token_ids = config.tokenizer.convert_tokens_to_ids(token)
    seq_len = len(token_ids)
    pad_size = config.pad_size
    if len(token_ids)<pad_size:
        token_ids = token_ids+[0]*(pad_size-len(token_ids))
        mask = [1]*seq_len+[0]*(pad_size-seq_len)
    else:
        token_ids = token_ids[:pad_size]
        seq_len = pad_size
        mask = [1]*seq_len
    input = (torch.LongTensor(token_ids).to(config.device),
             torch.LongTensor(seq_len).to(config.device),
             torch.LongTensor(mask).to(config.device))
    return input

In [71]:
input = load_data('中超-10人广州1-0击败10人河北 武汉三镇2-1上海海港',config)

In [72]:
label = config.class_list[torch.argmax(model(input))]

/tmp/ipykernel_112/337860286.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(out)


In [73]:
label

'sports'